In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def get_model_and_tokenizer(path):
    tokenizer = T5Tokenizer.from_pretrained(path)
    model = T5ForConditionalGeneration.from_pretrained(path, output_attentions=True)
    return tokenizer, model

In [ ]:
BASELINE_PATH = "baseline_seed1"
baseline_tokenizer, baseline_model = get_model_and_tokenizer(BASELINE_PATH)

# SQUAD_PATH = "squad_0.5_seed5"
# squad_tokenizer = T5Tokenizer.from_pretrained(SQUAD_PATH)
# squad_model = T5ForConditionalGeneration.from_pretrained(SQUAD_PATH, output_attentions=True)

# COMMONGEN_PATH = "commongen_0.1_seed4"
# commongen_tokenizer = T5Tokenizer.from_pretrained(COMMONGEN_PATH)
# commongen_model = T5ForConditionalGeneration.from_pretrained(COMMONGEN_PATH, output_attentions=True)

# COSMOS_PATH = "cosmos_1.0_seed9"
# cosmos_tokenizer, cosmos_model = get_model_and_tokenizer(COSMOS_PATH)


In [ ]:
def get_view(tokenzier, model, true_output):
    encoder_input_ids = tokenzier(INPUT_SENTENCE, return_tensors="pt", add_special_tokens=True).input_ids
    labels = tokenzier(true_output, return_tensors="pt", add_special_tokens=True).input_ids
    outputs = model(input_ids=encoder_input_ids, labels=labels)
    encoder_text = tokenzier.convert_ids_to_tokens(encoder_input_ids[0])

    generated_ids = model.generate(input_ids=encoder_input_ids, max_length=256, do_sample=True, top_p=0.9, top_k=0, num_return_sequences=1)
    preds = [tokenzier.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    print(preds[0])

    from bertviz import model_view, head_view
    head_view(
        encoder_attention=outputs.encoder_attentions,
        cross_attention=None,
        encoder_tokens= encoder_text,
        include_layers=[10,11]
    )

In [ ]:
INPUT_SENTENCE = "get sentiment: The Scene Hunky waiters dub diners darling and it sounds like they mean it. aspect: waiters"
OUTPUT = "negative"
get_view(baseline_tokenizer, baseline_model, OUTPUT)